In [3]:
from pyrealm.pmodel import (
    SubdailyScaler,
    memory_effect,
    SubdailyPModel,
    PModelEnvironment,
    PModel,
)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyrealm.pmodel.optimal_chi import OptimalChiPrentice14
from pyrealm.pmodel.functions import calc_ftemp_arrh, calc_ftemp_kphio,calc_modified_arrhenius_factor,calc_gammastar,calc_kmm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyrealm.constants import PModelConst


In [ ]:
# Read the target data table (including Date, Chamber, Temp_Treatment, etc.)
data = pd.read_csv("/GHS30_Euc2-SxTxCO2xW_GEtempresponse_20090105-20090109_L1.csv")

# Initialize result lists
gpp_results = []
anet_results = []
ci_results = []

# Additional outputs
ac_results = []
aj_results = []
vcmax_results = []
jmax_results = []
rd_results = []

# Constants
R = 8.3145
Tref = 298.15
ha_vcmax25 = 65330
ha_jmax25 = 43900

# Loop over each row in the dataset
for idx, row in data.iterrows():
    # 1. Extract acclimation environment inputs
    acclim_temp = row["temp_acclim"]
    co2_acclim = row["co2_acclim"]
    patm_acclim = row["patm_acclim"]
    vpd_acclim = row["vpd_acclim"]
    ppfd_acclim = row["light_acclim"]

    # 2. Build the acclimation environment and run the PModel
    env_acclim = PModelEnvironment(
        tc=np.array([acclim_temp]),
        patm=np.array([patm_acclim]),
        vpd=np.array([vpd_acclim]),
        co2=np.array([co2_acclim])
    )
    pmodel_acclim = PModel(
        env=env_acclim,
        method_kphio='temperature',
        reference_kphio=0.125,
        method_optchi="prentice14"
    )
    pmodel_acclim.estimate_productivity(fapar=1, ppfd=np.array([ppfd_acclim]))

    # 3. Calculate acclimated Vcmax25 and Jmax25
    vcmax25 = pmodel_acclim.vcmax[0] * np.exp((ha_vcmax25/R)*(1/(acclim_temp+273.15)-1/Tref))
    jmax25 = pmodel_acclim.jmax[0] * np.exp((ha_jmax25/R)*(1/(acclim_temp+273.15)-1/Tref))

    # 4. Extract subdaily environmental inputs
    tc = row["Tleaf"]
    patm = row['patm_acclim']
    vpd = row["VpdL"] * 1000   # Convert from kPa to Pa
    co2 = row["CO2S"]
    ppfd = row["PARi"]
    
    # 5. Build subdaily environment
    env_subdaily = PModelEnvironment(
        tc=np.array([tc]),
        patm=np.array([patm]),
        vpd=np.array([vpd]),
        co2=np.array([co2])
    )

    model = PModel(env_subdaily)

    # 6. Apply temperature response for Vcmax and Jmax
    vcmax = vcmax25 * np.exp((ha_vcmax25/R)*(1/Tref - 1/(tc + 273.15)))
    jmax = jmax25 * np.exp((ha_jmax25/R)*(1/Tref - 1/(tc + 273.15)))

    # 7. Estimate chi using the Prentice14 optimality model
    sub_chi = OptimalChiPrentice14(env=env_subdaily)
    xi = pmodel_acclim.optchi.xi
    sub_chi.estimate_chi(xi_values=xi)
    ci = sub_chi.ci

    # 8. Compute biochemical terms
    gammastar = calc_gammastar(tc, patm)
    kmm = calc_kmm(tc, patm)

    # Rubisco-limited assimilation
    ac = vcmax * (ci - gammastar) / (ci + kmm)

    # Electron-transport-limited assimilation
    fkphio = model.kphio.kphio
    J = (4 * fkphio * ppfd) / np.sqrt(1 + (4 * fkphio * ppfd / jmax) ** 2)
    aj = (J / 4) * ((ci - gammastar) / (ci + 2 * gammastar))

    # 9. Compute GPP and net photosynthesis
    gpp = min(ac[0], aj[0])
    rd = 0.03 * vcmax25
    anet = gpp - rd 

    # 10. Append results
    gpp_results.append(gpp)
    anet_results.append(anet)
    ci_results.append(ci[0])
    ac_results.append(ac[0])
    aj_results.append(aj[0])
    vcmax_results.append(vcmax)
    jmax_results.append(jmax)
    rd_results.append(rd)

# 11. Add model outputs to the dataframe
data["GPP_pmodel"] = np.clip(gpp_results, 0, None)
data["ci_pmodel"] = ci_results
data["anet_pmodel"] = anet_results

# 12. Add additional biochemical components
data["ac_pmodel"] = ac_results
data["aj_pmodel"] = aj_results
data["vcmax_pmodel"] = vcmax_results
data["jmax_pmodel"] = jmax_results
data["rd_pmodel"] = rd_results
